In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import cv2
from torchvision.utils import make_grid
import time

# 定义CNN模型
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.relu = nn.ReLU()
        self.target_layer = self.conv2

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 7*7*64)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义Grad-CAM类
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output

        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]

        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_full_backward_hook(backward_hook))

    def generate_cam(self, input_tensor, class_idx=None):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if class_idx is None:
            class_idx = torch.argmax(output, dim=1).item()
        target = output[0][class_idx]
        target.backward()

        gradients = self.gradients.detach().cpu().numpy()[0]
        activations = self.activations.detach().cpu().numpy()[0]

        weights = gradients.mean(axis=(1, 2), keepdims=True)
        cam = (weights * activations).sum(axis=0)

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (input_tensor.shape[2], input_tensor.shape[3]))
        cam = cam - np.min(cam)
        if np.max(cam) != 0:
            cam = cam / np.max(cam)
        else:
            print("Warning: max value of cam is 0, cannot normalize.")
            cam = np.zeros_like(cam)
        return cam

    def __del__(self):
        for handle in self.hook_handles:
            handle.remove()

# 设置数据加载器
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
model = CNNNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设置TensorBoard记录器
writer = SummaryWriter('runs/mnist_experiment')

# 添加模型结构到TensorBoard
data_iter = iter(train_loader)
images, labels = next(data_iter)
writer.add_graph(model, images)

# 训练模型
num_epochs = 5
for epoch in range(num_epochs):
    epoch_start_time = time.time()  
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            writer.add_scalar('training loss', running_loss / 100, epoch * len(train_loader) + i)
            running_loss = 0.0
    epoch_end_time = time.time()  
    epoch_duration = epoch_end_time - epoch_start_time  
    writer.add_scalar('epoch duration', epoch_duration, epoch)  
    
    # 记录每个epoch结束后的模型参数
    for name, param in model.named_parameters():
        writer.add_histogram(name, param, epoch)

    # 记录Grad-CAM的可视化结果
    data, _ = next(iter(train_loader))
    grad_cam = GradCAM(model, model.target_layer)
    cam = grad_cam.generate_cam(data[0].unsqueeze(0))

    if np.max(cam) != 0:
        cam = cam / (np.max(cam) + 0.001)
    else:
        print("Warning: max value of cam is 0, cannot normalize.")
        cam = np.zeros_like(cam)

    # 处理 cam 中的无效值
    cam[np.isnan(cam)] = 0
    cam[np.isinf(cam)] = 0

    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam_image = heatmap + np.repeat(np.expand_dims(np.float32(data[0].squeeze().cpu().numpy()), axis=-1), 3, axis=-1)
    cam_image = cam_image / (np.max(cam_image) + 0.001)

    # 转换为torch.Tensor并添加到TensorBoard
    cam_image_tensor = torch.tensor(cam_image).permute(2, 0, 1).unsqueeze(0)
    cam_image_tensor = (cam_image_tensor * 255).byte()
    writer.add_image('Grad-CAM', make_grid(cam_image_tensor), epoch)

writer.close()
